In [1]:
from pathlib import Path
modeltype = "gcnn"
version = "v08"
outdir = Path("../benchmarks/onnx/onnx_benchmark")
ckpt = Path(f"../saved_models/{modeltype}_{version}") # folder with checkpoints

In [2]:
# data loading
import numpy as np

# print(f"Extracting data into {outdir}...")
# !mkdir -p $outdir
# tar -xf dunetpc_inspired_v09_p2GeV_rawdigits.tar.gz -C $outdir

fname = outdir / "p2GeV_cosmics_inspired_rawdigit_evt8.npy"
evt = np.load(fname)
print(f"Loaded event at {fname}")

Loaded event at ../benchmarks/onnx/onnx_benchmark/p2GeV_cosmics_inspired_rawdigit_evt8.npy


In [3]:
# inference function
from time import time as tm

def inference(model, evt, fname):
    """Makes inference on event and computes time.

    Saves the output file to `fname`.

    Parameters
    ----------
    model: DnModel
        The pytorch or onnx based model.
    evt: np.ndarray
        The input raw data.
    fname: Path
        The output file name.
    
    Returns
    -------
    inference_time: float
        The elapsed time for inference.    
    """
    start = tm()
    evt_dn = model.inference(evt)
    inference_time = tm() - start

    # save pytorch inference
    np.save(fname, evt_dn)
    return inference_time

In [3]:
from dunedn.inference.hitreco import DnModel

# PyTorch model loading
model = DnModel(modeltype, ckpt)
print(f"Loaded model from {ckpt} folder")

Loaded model from ../../saved_models/gcnn_v08 folder


In [ ]:
# PyTorch inference
fname = outdir / "pytorch_inference_results.npy"
pytorch_time = inference(model, evt, fname)
print(f"PyTorch inference done in {pytorch_time}s")

In [ ]:
# uncomment this line to export model to ONNX format
model.export_onnx(outdir / f"saved_models/{modeltype}_{version}")

model_onnx = DnModel(
    modeltype,
    outdir / f"saved_models/{modeltype}_{version}",
    should_use_onnx=True
)

In [ ]:
# ONNX inference
fname = outdir / "onnx_inference_results.npy"
onnx_time = inference(model_onnx, evt, fname)
print(f"PyTorch inference done in {onnx_time}s")